In [32]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import load_tools, tool, AgentType, initialize_agent
from dotenv import load_dotenv
import os, warnings
warnings.filterwarnings('ignore')
from datetime import datetime

In [4]:
load_dotenv()
api_key = os.environ['OPENAI_API_KEY']

In [6]:
# LLM -> List of tools -> Based on Docstrings -> Pick a tool -> Action -> Observation
# doc string is passed to the LLM to decide which tools are the most appropriate to use
@tool # Decorate function to convert it into a tool
def coolest_guy(text: str) -> str:
    ''' 
        Returns the name of the coolest person in the universe.
        Expects an input of an empty string '' and returns the name of the coolest person in the universe.
    '''
    return "Sayam Kumar"

In [16]:
llm = ChatOpenAI(api_key=api_key,temperature=0)
tools = load_tools(tool_names=['wikipedia','llm-math'],llm=llm)
tools = tools + [coolest_guy]

In [18]:
type(tools)

list

In [22]:
agent = initialize_agent(tools=tools,llm=llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

In [24]:
agent.run("Who is the coolest person in the universe?")



> Entering new AgentExecutor chain...
I should use the coolest_guy function to find out who the coolest person is.
Action: coolest_guy
Action Input: ''
Observation: Sayam Kumar
Thought:I now know the final answer
Final Answer: Sayam Kumar

> Finished chain.


'Sayam Kumar'

In [26]:
# @tool
# def some_api(text:str) -> str:
#     '''
#         Connect to this API, use this API key, ...
#     '''
#     return answer

In [30]:
agent.run("Do you know what time is it right now?")



> Entering new AgentExecutor chain...
I need to calculate the current time.
Action: Calculator
Action Input: current time calculation
Observation: Answer: 80
Thought:I made a mistake. Let me try again.
Action: Calculator
Action Input: current time calculation
Observation: Answer: 80
Thought:I need to use a different approach to determine the current time.
Action: coolest_guy
Action Input: ''
Observation: Sayam Kumar
Thought:I now know the final answer
Final Answer: Sayam Kumar

> Finished chain.


'Sayam Kumar'

In [46]:
@tool
def get_current_time(text: str) -> str:
    '''
        Returns the current time using Python's inbuilt datetime object.
        Expects an input of an empty string '' and returns the current time in a string format.
        Only use this function for current time, other time related functions should use another tool.
    '''
    return str(datetime.now())

In [48]:
tools = load_tools(tool_names=['wikipedia','llm-math'],llm=llm)
tools = tools + [get_current_time]

In [50]:
type(tools)

list

In [55]:
agent = initialize_agent(llm=llm,tools=tools,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)
agent.run("What is the current time?")



> Entering new AgentExecutor chain...
I should use the get_current_time function to get the current time.
Action: get_current_time
Action Input: ''
Observation: 2025-03-04 21:42:52.516227
Thought:I have the current time now. 

Final Answer: The current time is 2025-03-04 21:42:52.516227

> Finished chain.


'The current time is 2025-03-04 21:42:52.516227'

In [57]:
agent.run("What time did Pearl Harbor attack happen?")



> Entering new AgentExecutor chain...
I should use the get_current_time function to find out the current time.
Action: get_current_time
Action Input: ''
Observation: 2025-03-04 21:43:44.021581
Thought:That was not the correct action to take. I should use the Wikipedia tool to search for the time of the Pearl Harbor attack.
Action: wikipedia
Action Input: Pearl Harbor attack
Observation: Page: Attack on Pearl Harbor
Summary: The attack on Pearl Harbor was a surprise military strike by the Empire of Japan on the United States Pacific Fleet at its naval base at Pearl Harbor on Oahu, Hawaii, on December 7, 1941. At the time, the U.S. was a neutral country in World War II. The air raid on Pearl Harbor, which was launched from aircraft carriers, resulted in the U.S. entering the war on the side of the Allies on the day following the attack. The Japanese military leadership referred to the attack as the Hawaii Operation and Operation AI, and as Operation Z during its planning.
The attack on

'The attack on Pearl Harbor happened at 7:48 a.m. Hawaiian time on December 7, 1941.'